<a href="https://colab.research.google.com/github/sarajay19/T5_W6/blob/main/YOLO_Practical_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [ ]:
!pip install opencv-python ultralytics

## Load the YOLO Model

You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from ultralytics import YOLO

# loading the YOLO model from thr drive
model = YOLO('/content/drive/MyDrive/Trainees - المتدربين/Week 6/6- Introduction to computer vision/5- Weekly Project (Sunday)/Dataset/YOLO_Model.pt')

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np

In [ ]:
# loading the video and sign it to the video path
video_path = '/content/drive/MyDrive/Trainees - المتدربين/Week 6/6- Introduction to computer vision/5- Weekly Project (Sunday)/Dataset/YOLO_Video.mp4'
cap = cv2.VideoCapture(video_path)

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [ ]:
# extracting the info from video
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = cap.get(cv2.CAP_PROP_FPS)

In [ ]:
# print out each info to use on th roi later
print(f"Video Height: {height}")
print(f"Video Width: {width}")
print(f"Video FPS: {fps}")

## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [ ]:
output_path = 'output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [ ]:
# define roi box for counting
roi = [
    (1050, 0, 1200, 350),  # top
    (700, height - 300, 800, height - 0),  # bottom
    (0, 350, 500, 500),  # left
    (width - 700, 550, width - 0, 700)  # right
]

In [ ]:
# counters for each exit
exit_counters = [0, 0, 0, 0]  # top, bottom, left, right

## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

In [ ]:
vehicle_ids = set()  # to track vehichle IDs

In [13]:
# process video frames
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("Failed to read frame")
        break

    # detection
    results = model(frame)

    # store current frame vehichle ID
    current_vehicle_ids = set()

    for result in results:
        for detection in result.boxes.data:  # iterate through boxes
            x1, y1, x2, y2, conf, cls = detection.tolist()

            # Confidence threshold
            if conf < 0.7:  # adjusting the threshold
                continue

            # assign unique ID
            vehicle_id = int(cls)  # Assuming class ID is used as vehicle ID

            # add the vehicle ID to the current frame
            current_vehicle_ids.add(vehicle_id)

            # Drawing the bounding box
            cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)

            # Check which ROI exited
            if y2 < roi[0][1]:  # Top
                exit_counters[0] += 1
            elif y1 > roi[1][3]:  # Bottom
                exit_counters[1] += 1
            elif x2 < roi[2][0]:  # Left
                exit_counters[2] += 1
            elif x1 > roi[3][2]:  # Right
                exit_counters[3] += 1

            # Display conf score, IDs
            label = f"ID: {vehicle_id}, Conf: {conf:.2f}"
            cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

            # calculate the center of the vehichle
            center_x = int((x1 + x2) / 2)
            center_y = int((y1 + y2) / 2)
            cv2.circle(frame, (center_x, center_y), 5, (255, 0, 0), -1)  # Center point

    # Update vehicle IDs for the next frame
    vehicle_ids.update(current_vehicle_ids)

    # drawing the roi
    for (x1, y1, x2, y2) in roi:
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)  # red rectangle for the roi

    # display counts
    cv2.putText(frame, f"Top: {exit_counters[0]}", (200, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    cv2.putText(frame, f"Bottom: {exit_counters[1]}", (200, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    cv2.putText(frame, f"Left: {exit_counters[2]}", (200, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    cv2.putText(frame, f"Right: {exit_counters[3]}", (200, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)

    # displaying my name
    cv2.putText(frame, "Sara", (200, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

    # showing the frame
    cv2_imshow(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # breaking the loop
        break

    # write the frame to the output video
    out.write(frame)

# release resources
cap.release()
out.release()
cv2.destroyAllWindows()

## Save and Submit Your Work

Submit both the notebook and the output video

Resources used:

https://yolov8.org/yolov8-documentation/#5_Training


https://opencv-tutorial.readthedocs.io/en/latest/yolo/yolo.html
